In [32]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.regularizers import l2

In [33]:
from PIL import Image
import numpy as np

In [41]:
def load2000(path):
    N= 2000
    N_split= int(N*0.7)
    
    l=[]
    for name in range(N):
        filename= path + str(name) + '.png'
        img = Image.open( filename )
        data16 = np.asarray( img, dtype="int16" )
        l.append(data16)

    x= np.stack(l)
    
    # np.savetxt('data/labels.txt', y_train[:2000], fmt="%d")
    y = np.loadtxt("data/labels.txt", dtype='int16')
    
    x_train= x[: N_split ]
    y_train= y[: N_split ]

    x_test= x[ N_split :]
    x_test= y[ N_split :]
    
    return (x_train, y_train), (x_test, x_test)

(x_train, y_train), (x_test, y_test)= load2000('data/')
print(type(x_train))
print( x_train.shape)
print( type(y_train ) )
print( y_train.shape )

<class 'numpy.ndarray'>
(1400, 28, 28)
<class 'numpy.ndarray'>
(1400,)


In [42]:
# with mirrored_strategy.scope():
with tf.device('/cpu:0'):
#     mnist = tf.keras.datasets.mnist
#     (x_train, y_train), (x_test, y_test) = mnist.load_data()
#     x_train, x_test = x_train / 255.0, x_test / 255.0

    print( type(x_train), type(y_train ) )
    print(x_train.shape, y_train.shape)
    
    img_rows, img_cols= 28, 28
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
    
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(16, kernel_size=(3, 3)),
        tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer='adam',
                  loss=loss_fn,
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=1)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(1400, 28, 28) (1400,)
Train on 1400 samples
1400/1400 [==============================] - 1s 892us/sample - loss: 7.8523 - accuracy: 0.5743
